In [30]:
import sqlite3
import datetime
import json
from typing import List, Dict, Any, Optional

def get_chat_mapping(db_location):
    conn = sqlite3.connect(db_location)
    cursor = conn.cursor()
    cursor.execute("SELECT ROWID, room_name, display_name FROM chat")
    result_set = cursor.fetchall()
    # Include the ROWID (chat_id) in the mapping
    mapping = {row[0]: {"room_name": row[1], "display_name": row[2]} for row in result_set}
    conn.close()
    return mapping

def read_messages(db_location, n=10, self_number='Me', human_readable_date=True):
    conn = sqlite3.connect(db_location)
    cursor = conn.cursor()
    
    # Modified query to include chat_id from the chat_message_join table
    query = """
    SELECT 
        message.ROWID, 
        message.date, 
        message.text, 
        message.attributedBody, 
        handle.id, 
        message.is_from_me, 
        message.cache_roomnames,
        cmj.chat_id  -- Added chat_id from chat_message_join
    FROM message
    LEFT JOIN handle ON message.handle_id = handle.ROWID
    LEFT JOIN chat_message_join cmj ON message.ROWID = cmj.message_id
    """
    
    if n is not None:
        query += f" ORDER BY message.date DESC LIMIT {n}"
    
    results = cursor.execute(query).fetchall()
    messages = []
    
    chat_mapping = get_chat_mapping(db_location)
    
    for result in results:
        rowid, date, text, attributed_body, handle_id, is_from_me, cache_roomname, chat_id = result
        
        if handle_id is None:
            phone_number = self_number
        else:
            phone_number = handle_id
            
        if text is not None:
            body = text
        elif attributed_body is None:
            continue
        else:
            # Handle attributed body decoding
            attributed_body = attributed_body.decode('utf-8', errors='replace')
            if "NSNumber" in str(attributed_body):
                attributed_body = str(attributed_body).split("NSNumber")[0]
                if "NSString" in attributed_body:
                    attributed_body = str(attributed_body).split("NSString")[1]
                    if "NSDictionary" in attributed_body:
                        attributed_body = str(attributed_body).split("NSDictionary")[0]
                        attributed_body = attributed_body[6:-12]
                        body = attributed_body
        
        if human_readable_date:
            date_string = '2001-01-01'
            mod_date = datetime.datetime.strptime(date_string, '%Y-%m-%d')
            unix_timestamp = int(mod_date.timestamp())*1000000000
            new_date = int((date+unix_timestamp)/1000000000)
            date = datetime.datetime.fromtimestamp(new_date).strftime("%Y-%m-%d %H:%M:%S")
        
        # Get chat information if chat_id exists
        chat_info = None
        if chat_id is not None and chat_id in chat_mapping:
            chat_info = chat_mapping[chat_id]
            mapped_name = chat_info.get("display_name")
            room_name = chat_info.get("room_name")
        else:
            mapped_name = None
            room_name = cache_roomname
        
        messages.append({
            "rowid": rowid, 
            "date": date, 
            "body": body, 
            "phone_number": phone_number, 
            "is_from_me": is_from_me,
            "cache_roomname": cache_roomname, 
            'group_chat_name': mapped_name,
            'chat_id': chat_id  # Added chat_id to the output
        })
    
    conn.close()
    return messages

def print_messages(messages):
    for message in messages:
        print(f"RowID: {message['rowid']}")
        print(f"Chat ID: {message['chat_id']}")  # Added chat_id to the output
        print(f"Body: {message['body']}")
        print(f"Phone Number: {message['phone_number']}")
        print(f"Is From Me: {message['is_from_me']}")
        print(f"Cache Roomname: {message['cache_roomname']}")
        print(f"Group Chat Name: {message['group_chat_name']}")
        print(f"Date: {message['date']}")
        print("\n")

def group_messages_by_conversation(messages):
    """Group messages by conversation ID"""
    conversations = {}
    
    for message in messages:
        chat_id = message['chat_id']
        if chat_id is None:
            # Skip messages without a chat_id
            continue
            
        if chat_id not in conversations:
            conversations[chat_id] = {
                'chat_id': chat_id,
                'group_chat_name': message['group_chat_name'],
                'messages': []
            }
            
        conversations[chat_id]['messages'].append(message)
    
    # Sort conversations by the date of the most recent message
    for chat_id in conversations:
        conversations[chat_id]['messages'].sort(key=lambda x: x['date'])
        
    return conversations

In [31]:
import os
db_location = "~/Library/Messages/chat.db"
# Read most recent 1000 messages
db_location = os.path.expanduser(db_location)
messages = read_messages(db_location, n=25000)

# Group them by conversation
conversations = group_messages_by_conversation(messages)

# Print conversation statistics
print(f"Found {len(conversations)} conversations")
    

Found 102 conversations


In [32]:
# Print a summary of each conversation
for chat_id, convo in conversations.items():
    print(f"Conversation: {convo['group_chat_name'] or 'Unknown'} (ID: {chat_id})")
    print(f"  Message count: {len(convo['messages'])}")
    if convo['messages']:
        first_msg = convo['messages'][0]['date']
        last_msg = convo['messages'][-1]['date']
        print(f"  Date range: {first_msg} to {last_msg}")
    print()

Conversation: Unknown (ID: 466)
  Message count: 54
  Date range: 2025-04-22 23:25:36 to 2025-05-07 01:29:31

Conversation: Unknown (ID: 58)
  Message count: 57
  Date range: 2025-04-08 03:33:44 to 2025-05-07 01:25:24

Conversation: Unknown (ID: 315)
  Message count: 434
  Date range: 2025-04-17 19:32:54 to 2025-05-06 22:58:14

Conversation: Aquatic Avian Kids (ID: 37)
  Message count: 334
  Date range: 2025-04-08 05:43:43 to 2025-05-06 22:38:37

Conversation: Unknown (ID: 7)
  Message count: 102
  Date range: 2025-04-09 00:15:37 to 2025-05-06 21:01:54

Conversation: Book Report Club - TEAM BOOKWORMS (ID: 339)
  Message count: 30
  Date range: 2025-04-07 06:18:57 to 2025-05-06 19:06:23

Conversation: Groussangerpeltz Chat (ID: 357)
  Message count: 135
  Date range: 2025-04-06 20:20:51 to 2025-05-06 18:33:22

Conversation: Unknown (ID: 104)
  Message count: 7
  Date range: 2025-05-06 17:29:47 to 2025-05-06 17:38:44

Conversation: Unknown (ID: 16)
  Message count: 123
  Date range: 2025